# Geolocalizando endereços

In [118]:
import pyodbc
import pandas as pd

server = 'DCSRVDSDBBI' 
database = 'DW_ESUSVS' 
username = 'DW_Prodest' 
password = 'MEgt9q1R4b'  
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

query = "SELECT top 1000 * FROM DimCidadao order by cidadao_id desc;"
df = pd.read_sql(query, cnxn)

df["endereco"]

df.head()

,CidadaoId,CidadaoCod,cartao_sus,nome,idade,idade_ms,nome_mae,data_nascimento,SexoCod,Sexo,...,raca_cor,cidadao_id,distrito,pais,territorio_id,endereco_outra_cidade,etnia,has_deficiencia,eh_morador_rua,DataInclusao
0,144465,536044,44bc28c4c847dbd844e6b73832c477a58182a734,c64a3e5a496f8b773ef0d313ddf8308c45788313,24 anos e 6 meses,4024,None,1996-03-28,M,Masculino,...,Parda,None,,BRASIL,None,,None,Não,Não,2021-05-12 17:04:10.743
1,1684,590410,c2d7f2066e21f27c02ae372a3dd9c3065f0c09c1,b90363d6e509d81f8a61a3fad7346444ecc15abe,"47 anos, 2 meses, e 5 dias",4047,None,1973-08-07,M,Masculino,...,Ignorado,None,,BRASIL,None,NÃO INFORMADO,None,Não,Não,2021-05-12 17:04:10.743
2,144466,536045,b89d93bfb29b456e106dba3d6c8a0942d341c68f,5b80582912f4716ac88bef7fc224a944dd34f933,"23 anos, 9 meses, e 9 dias",4023,None,1996-12-19,M,Masculino,...,Amarela,None,,BRASIL,None,PR SM,None,Não,Não,2021-05-12 17:04:10.743
3,1685,590411,4b599179b0100491d3d9bdcb8ac3bf825ec51de9,77eecd4b5cf37533cf9943e6bb868c00d1882593,"58 anos, 6 meses, e 21 dias",4058,None,1962-03-22,F,Feminino,...,Ignorado,None,,BRASIL,None,,None,Não,Não,2021-05-12 17:04:10.743
4,144467,536046,eda16d299eb72e748aa19caf74430e5d62022a84,09b6b80f7d8442c1bedef73688dbf80ade9290f4,"37 anos, 11 meses, e 7 dias",4037,None,1982-10-21,F,Feminino,...,Preta,None,,BRASIL,None,,None,Não,Não,2021-05-12 17:04:10.743


In [119]:
# https://www.youtube.com/watch?v=asSNKH_ee0M
# https://geopy.readthedocs.io/en/stable/index.html?highlight=rate%20limiter#module-geopy.extra.rate_limiter

# importa a biblioteca que faz requisições para um serviço de geolocalização de endereços
from geopy.geocoders import Nominatim

# testando para um endereço específico
locator = Nominatim(user_agent="MyGeocoder")
location = locator.geocode("Rua Castelo, 100, Jardim Limoeiro, Serra, Espírito Santo")

In [120]:
location.point

Point(-20.2092599, -40.2637956, 0.0)

In [121]:
location.address

'Rua Castelo, Jardim Limoeiro, Carapina, Serra, Região Geográfica Imediata de Vitória, Região Metropolitana da Grande Vitória, Região Geográfica Intermediária de Vitória, Espírito Santo, Região Sudeste, 29164-018, Brasil'

In [122]:
# para fazer para várias linhas precisamos colocar um rate limiter (limite de tempo entre as requisições)
# https://geopy.readthedocs.io/en/stable/index.html?highlight=rate%20limiter#module-geopy.extra.rate_limiter

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

In [123]:
df['geo_location'] = df['endereco'].apply(geocode)

In [124]:
df['point'] = df['geo_location'].apply(lambda loc: tuple(loc.point) if loc else None)
df["lat"] = df['point'].apply(lambda point: tuple(point)[0] if point else None)
df["long"] = df['point'].apply(lambda point: tuple(point)[1] if point else None)

dados = df[["endereco", "geo_location", "point", "lat", "long"]]
dados

,endereco,geo_location,point,lat,long
0,CRUZEIRO DO SUL R. DO VALE 03 CASA,None,None,NaN,NaN
1,AEROPORTO Não Encontrado,None,None,NaN,NaN
2,RANCHO DO TELHA Não Encontrado,None,None,NaN,NaN
3,RESIDENCIAL JACARAÍPE R. SANDRA RIBEIRO 33 CASA,None,None,NaN,NaN
4,BOA VISTA II ESC. RAUL POMPÉIA 1 CASA,None,None,NaN,NaN
...,...,...,...,...,...
995,CENTRO R. ELDA MENDONÇA DE LIMA 17 CASA,None,None,NaN,NaN
996,SÃO JOSÉ R. PRIMEIRO DE MAIO 70 CASA,None,None,NaN,NaN
997,PESQUEIRO Não Encontrado,None,None,NaN,NaN
998,MELGACINHO Não Encontrado,None,None,NaN,NaN


In [125]:
import plotly.express as px

df = px.data.carshare()
fig = px.scatter_mapbox(dados, lat="lat", lon="long",  
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10,
                  mapbox_style="carto-positron")
fig.show()